In [18]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, norm
import matplotlib.pyplot as plt
import seaborn as sns
from six import print_


# Conditional Outcome Function
def conditional_outcome(s1, s2, result, sigma_t):
    mean_diff = s1 - s2
    std_diff = sigma_t

    if result == 1:
        a, b = 0, np.inf  
    else:
        a, b = -np.inf, 0  

    a_std = (a - mean_diff) / std_diff
    b_std = (b - mean_diff) / std_diff

    return a_std, b_std, mean_diff, std_diff

def gibbs_sampler_result(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t, n_iter, result):
    s1_samples = np.zeros(n_iter)
    s2_samples = np.zeros(n_iter)
    t_samples = np.zeros(n_iter)
    y_samples = np.zeros(n_iter)
    
    s1_current = mu_s1
    s2_current = mu_s2

    mu_prior = np.array([mu_s1, mu_s2]).reshape((2, 1))
    sigma_prior = np.array([[sigma_s1, 0], [0, sigma_s2]])
    sigma_prior_inv = np.linalg.inv(sigma_prior)

    A = np.array([1, -1]).reshape((1, 2))
    A_T = A.T

    for i in range(n_iter):
        a_std, b_std, mean_diff, std_diff = conditional_outcome(s1_current, s2_current, result, sigma_t)
        t_current = truncnorm.rvs(a=a_std, b=b_std, loc=mean_diff, scale=std_diff)
        sigma_post = np.linalg.inv(sigma_prior_inv + (A_T @ A) / sigma_t**2)
        mu_post = sigma_post @ (sigma_prior_inv @ mu_prior + (A_T * t_current) / sigma_t**2)
        s1_current, s2_current = np.random.multivariate_normal(mu_post.flatten(), sigma_post)
        s1_samples[i] = s1_current
        s2_samples[i] = s2_current
        t_samples[i] = t_current
        y_samples[i] = result

    mean_s1 = np.mean(s1_samples)
    var_s1 = np.var(s1_samples, ddof=1)
    mean_s2 = np.mean(s2_samples)
    var_s2 = np.var(s2_samples, ddof=1)

    return mean_s1, var_s1, mean_s2, var_s2

def win_probability_player1(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t):
    mu_diff = mu_s1 - mu_s2
    sigma_diff = np.sqrt(sigma_s1 + sigma_s2 + sigma_t**2)
    p_y1 = 1 - norm.cdf(0, loc=mu_diff, scale=sigma_diff)
    return p_y1




In [42]:
data_file = 'SerieA.csv'
df = pd.read_csv(data_file)
df_filtered = df[df['score1'] != df['score2']].copy()
df_filtered['win'] = df_filtered.apply(lambda row: 1 if row['score1'] > row['score2'] else -1, axis=1)
initial_mu = 25
initial_sigma = 8.33
sigma_t = 25 / 6
teams = set(df_filtered['team1']).union(set(df_filtered['team2']))
skills = {team: (initial_mu, initial_sigma) for team in teams}
n_iter = 100 
predictions = []
random_predictions = []



for index, row in df_filtered.iterrows():
    team1 = row['team1']
    team2 = row['team2']
    result = row['win']

    # Get prior means and variances
    mu_s1, sigma_s1 = skills[team1]
    mu_s2, sigma_s2 = skills[team2]

    # Run Gibbs sampler
    mean_s1, var_s1, mean_s2, var_s2 = gibbs_sampler_result(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t, n_iter, result)

    # Update skills: Update both mean and variance
    skills[team1] = (mean_s1, var_s1)
    skills[team2] = (mean_s2, var_s2)
    
    if index < 272:
        p_y1 = win_probability_player1(mu_s1, mu_s2, sigma_s1, sigma_s2, sigma_t)
        predictions.append(1 if p_y1 > 0.5 else -1)
        random_predictions.append(1 if np.random.rand() > 0.5 else -1)

correct = 0
correct_random = 0

for i in range(len(predictions)):
    if predictions[i] == df_filtered.iloc[i]['win']:
        correct += 1
    if random_predictions[i] == df_filtered.iloc[i]['win']:
        correct_random += 1

print(f"Accuracy of the model: {correct / len(predictions)}")
print(f"Accuracy of the random model: {correct_random / len(predictions)}")

Accuracy of the model: 0.6804123711340206
Accuracy of the random model: 0.4742268041237113


In [43]:
# Sort and Display the Updated Skills
# Sort teams by their updated mean skills in descending order
sorted_skills = sorted(skills.items(), key=lambda x: x[1][0], reverse=True)

print("\nUpdated Skills:")
for team, (mu, sigma) in sorted_skills:
    print(f"{team}: Mean Skill = {mu:.2f}, Variance = {sigma:.2f}")




Updated Skills:
Juventus: Mean Skill = 29.14, Variance = 2.22
Roma: Mean Skill = 27.26, Variance = 0.34
Inter: Mean Skill = 27.16, Variance = 1.22
Napoli: Mean Skill = 27.08, Variance = 0.85
Milan: Mean Skill = 26.96, Variance = 0.75
Torino: Mean Skill = 26.55, Variance = 0.85
Atalanta: Mean Skill = 26.06, Variance = 0.79
Lazio: Mean Skill = 25.72, Variance = 1.16
Sampdoria: Mean Skill = 24.14, Variance = 1.66
Genoa: Mean Skill = 23.73, Variance = 1.40
Sassuolo: Mean Skill = 23.68, Variance = 1.18
Spal: Mean Skill = 23.62, Variance = 0.54
Parma: Mean Skill = 23.57, Variance = 0.51
Bologna: Mean Skill = 23.29, Variance = 0.52
Cagliari: Mean Skill = 23.15, Variance = 1.41
Empoli: Mean Skill = 22.99, Variance = 1.07
Udinese: Mean Skill = 22.84, Variance = 0.79
Fiorentina: Mean Skill = 22.50, Variance = 1.27
Frosinone: Mean Skill = 21.02, Variance = 1.18
Chievo: Mean Skill = 20.25, Variance = 1.13
